# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2022 1:28PM,237884,16,SHL-8053108,"SHL Pharma, LLC",Released
1,May 23 2022 1:23PM,237881,19,DEX0006846,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
2,May 23 2022 1:23PM,237881,19,DEX0006847,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
3,May 23 2022 1:23PM,237881,19,DEX0006848,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
4,May 23 2022 1:23PM,237881,19,DEX0006849,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
5,May 23 2022 1:23PM,237881,19,DEX0006850,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
6,May 23 2022 1:23PM,237881,19,DEX0006851,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
7,May 23 2022 1:21PM,237878,20,8051169,Reliable 1 Laboratories LLC,Released
8,May 23 2022 1:21PM,237878,20,8051170,Reliable 1 Laboratories LLC,Released
9,May 23 2022 1:21PM,237878,20,8051171,Reliable 1 Laboratories LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,237877,Released,1
37,237878,Released,21
38,237880,Released,2
39,237881,Released,6
40,237884,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237877,NaN,NaN,1.0
237878,NaN,NaN,21.0
237880,NaN,NaN,2.0
237881,NaN,NaN,6.0
237884,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0.0,0.0,1.0
237794,0.0,13.0,13.0
237795,0.0,0.0,1.0
237796,0.0,1.0,0.0
237804,11.0,5.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0,0,1
237794,0,13,13
237795,0,0,1
237796,0,1,0
237804,11,5,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,0,0,1
1,237794,0,13,13
2,237795,0,0,1
3,237796,0,1,0
4,237804,11,5,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,,,1
1,237794,,13,13
2,237795,,,1
3,237796,,1,
4,237804,11,5,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2022 1:28PM,237884,16,"SHL Pharma, LLC"
1,May 23 2022 1:23PM,237881,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
7,May 23 2022 1:21PM,237878,20,Reliable 1 Laboratories LLC
28,May 23 2022 1:21PM,237880,10,Emerginnova
30,May 23 2022 1:17PM,237877,19,ACG North America LLC
31,May 23 2022 1:02PM,237875,10,"Snap Medical Industries, LLC"
44,May 23 2022 12:41PM,237872,19,"AdvaGen Pharma, Ltd"
51,May 23 2022 12:34PM,237869,10,ISDIN Corporation
59,May 23 2022 12:25PM,237867,22,"NBTY Global, Inc."
60,May 23 2022 12:24PM,237866,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2022 1:28PM,237884,16,"SHL Pharma, LLC",,,1
1,May 23 2022 1:23PM,237881,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,6
2,May 23 2022 1:21PM,237878,20,Reliable 1 Laboratories LLC,,,21
3,May 23 2022 1:21PM,237880,10,Emerginnova,,,2
4,May 23 2022 1:17PM,237877,19,ACG North America LLC,,,1
5,May 23 2022 1:02PM,237875,10,"Snap Medical Industries, LLC",,,13
6,May 23 2022 12:41PM,237872,19,"AdvaGen Pharma, Ltd",,,7
7,May 23 2022 12:34PM,237869,10,ISDIN Corporation,,,8
8,May 23 2022 12:25PM,237867,22,"NBTY Global, Inc.",,,1
9,May 23 2022 12:24PM,237866,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 1:28PM,237884,16,"SHL Pharma, LLC",1,,
1,May 23 2022 1:23PM,237881,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,6,,
2,May 23 2022 1:21PM,237878,20,Reliable 1 Laboratories LLC,21,,
3,May 23 2022 1:21PM,237880,10,Emerginnova,2,,
4,May 23 2022 1:17PM,237877,19,ACG North America LLC,1,,
5,May 23 2022 1:02PM,237875,10,"Snap Medical Industries, LLC",13,,
6,May 23 2022 12:41PM,237872,19,"AdvaGen Pharma, Ltd",7,,
7,May 23 2022 12:34PM,237869,10,ISDIN Corporation,8,,
8,May 23 2022 12:25PM,237867,22,"NBTY Global, Inc.",1,,
9,May 23 2022 12:24PM,237866,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 1:28PM,237884,16,"SHL Pharma, LLC",1,,
1,May 23 2022 1:23PM,237881,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,6,,
2,May 23 2022 1:21PM,237878,20,Reliable 1 Laboratories LLC,21,,
3,May 23 2022 1:21PM,237880,10,Emerginnova,2,,
4,May 23 2022 1:17PM,237877,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 1:28PM,237884,16,"SHL Pharma, LLC",1.0,NaN,NaN
1,May 23 2022 1:23PM,237881,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,6.0,NaN,NaN
2,May 23 2022 1:21PM,237878,20,Reliable 1 Laboratories LLC,21.0,NaN,NaN
3,May 23 2022 1:21PM,237880,10,Emerginnova,2.0,NaN,NaN
4,May 23 2022 1:17PM,237877,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 1:28PM,237884,16,"SHL Pharma, LLC",1.0,0.0,0.0
1,May 23 2022 1:23PM,237881,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,6.0,0.0,0.0
2,May 23 2022 1:21PM,237878,20,Reliable 1 Laboratories LLC,21.0,0.0,0.0
3,May 23 2022 1:21PM,237880,10,Emerginnova,2.0,0.0,0.0
4,May 23 2022 1:17PM,237877,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2854096,262.0,69.0,11.0
12,475700,2.0,1.0,0.0
15,237794,13.0,13.0,0.0
16,237884,1.0,0.0,0.0
18,1664541,6.0,1.0,0.0
19,951482,14.0,1.0,0.0
20,951441,81.0,7.0,14.0
22,475733,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2854096,262.0,69.0,11.0
1,12,475700,2.0,1.0,0.0
2,15,237794,13.0,13.0,0.0
3,16,237884,1.0,0.0,0.0
4,18,1664541,6.0,1.0,0.0
5,19,951482,14.0,1.0,0.0
6,20,951441,81.0,7.0,14.0
7,22,475733,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,262.0,69.0,11.0
1,12,2.0,1.0,0.0
2,15,13.0,13.0,0.0
3,16,1.0,0.0,0.0
4,18,6.0,1.0,0.0
5,19,14.0,1.0,0.0
6,20,81.0,7.0,14.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,262.0
1,12,Released,2.0
2,15,Released,13.0
3,16,Released,1.0
4,18,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,22
Status,,,,,,,,
Completed,11.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0
Executing,69.0,1.0,13.0,0.0,1.0,1.0,7.0,0.0
Released,262.0,2.0,13.0,1.0,6.0,14.0,81.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,22
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0
1,Executing,69.0,1.0,13.0,0.0,1.0,1.0,7.0,0.0
2,Released,262.0,2.0,13.0,1.0,6.0,14.0,81.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,22
0,Completed,11.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0
1,Executing,69.0,1.0,13.0,0.0,1.0,1.0,7.0,0.0
2,Released,262.0,2.0,13.0,1.0,6.0,14.0,81.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()